# Pruebas

## Voxel size

In [ ]:
import os
import numpy as np
import nibabel as nb
from skimage.measure import regionprops, label
from matplotlib import pyplot as plt
import math

METHOD = 'reg' # reg, dl, manual
LABEL = 'lens+globe' # eyeball, lens+globe

# Paths - segmentation results
if METHOD=='reg':
    main_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/'
    input_path = main_path + 'non_labeled_dataset_nifti_reg_2/'
elif METHOD=='dl':
    main_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/3D_multilabel/experiment_0/'
    input_path = main_path + 'test_orig_reg-cropped_non-labeled/'
elif METHOD=='manual':
    main_path = '/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/'
    input_path = main_path + 'a123/'

# List of volumes for each individual label and subject
len_path = len([elem for elem in os.listdir(input_path)])
name_subject = [None]*len_path
axial_length_total = np.zeros(len_path)
outliers = list()

i=0
for folder1 in sorted(os.listdir(input_path)):

    # Load image in array format
    if METHOD=='reg': lab_arr = nb.load(f'{input_path}{folder1}/labels.nii.gz').header.get_zooms()
    elif METHOD=='dl': lab_arr = nb.load(f'{input_path}{folder1}').get_fdata()
    elif METHOD=='manual': lab_arr = nb.load(f'{input_path}{folder1}/input/{folder1}_labels_cropped.nii.gz').get_fdata()

    print(f'voxel size = ({lab_arr})')

    i+=1
    if i==1:
        break

## Sublist from list

In [ ]:
import SimpleITK as sitk
import sys, os, glob
import pydicomext as pydcm
from pydicom.fileset import FileSet
from deid.data import get_dataset
from deid.dicom import get_files, get_identifiers
from pathlib import Path
import numpy as np

input_path = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset/'

# List of volumes for each individual label and subject
len_path = len([elem for elem in os.listdir(input_path)])
name_subject = [None]*len_path
age_subject = [None]*len_path
sex_subject = [None]*len_path
height_subject = [None]*len_path
weight_subject = [None]*len_path
bmi_subject = [None]*len_path

# Create sorted list
i=0
list_subs = [None]*len_path
for folder1 in sorted(os.listdir(input_path)):
    list_subs[i] = folder1
    i+=1

index_sub = list_subs.index('2022160103683')
print(index_sub)

# Loop sublist
k=0
for j in range(index_sub, len_path):
    k+=1
print(k)    

## itkwidgets

imports and paths

In [1]:
import nibabel as nb
import numpy as np
import scipy.misc
import itk, itkwidgets
from itkwidgets import view

ima_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/non_labeled_dataset_nifti_nnunet/'
lab_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing/'
ima_file = 'AEye_2022160100001_0000.nii.gz'
lab_file = 'AEye_2022160100001.nii.gz'

nibabel

In [ ]:
# ITKWIDGETS
# view(ima_arr[:, :, slices[0]:slices[len(slices)-1]])
# ima_itk = itk.imread(f'{input_path}{folder1}')
# view(ima_arr_read)


# ima_nb = nb.load(f'{ima_path}{ima_file}')
# ima_arr = np.array(ima_nb.get_fdata()) # image matrix
# ima_arr_uint8 = ima_arr.astype(np.uint8)

# view(ima_arr)

lab_nb = nb.load(f'{lab_path}{lab_file}')
lab_arr = np.array(lab_nb.get_fdata()) # image matrix
lab_arr_uint8 = lab_arr.astype(np.uint8)

view(lab_arr)

itk

In [2]:
lab_itk = itk.imread(f'{lab_path}{lab_file}')

view(lab_itk)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [ ]:
# Segment the cells
smoothed = itk.smoothing_recursive_gaussian_image_filter(lab_itk)

thresholded = itk.moments_threshold_image_filter(smoothed,
                                                 inside_value=0,
                                                 outside_value=1)

connected_components = itk.connected_component_image_filter(thresholded)

In [ ]:
view(label_image=connected_components)